<a href="https://colab.research.google.com/github/lphansen/Twocap/blob/main/Twocap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sluggish Two Capital Model

In [1]:
using Pkg
using Optim
using Roots
using NPZ
using Distributed
using CSV
using Tables
using ArgParse
using Interpolations

# 1 Model

## 1.1 Exogenous State Evolution

We presume that there are two underlying exogenous processes that evolve as solutions to stochastic dfferential equations

$$
\begin{align}
& d Z_t^1=-\mathrm{a}_{11} Z_t^1 d t+\sqrt{Z_t^2} \sigma_1 \cdot d B_t \\
& d Z_t^2=-\mathrm{a}_{22}\left(Z_t^2-1\right) d t+\sqrt{Z_t^2} \sigma_2 \cdot d B_t
\end{align}
$$

where $\mathrm{a}_{11}>0, \mathrm{a}_{22}>\frac{1}{2}\left|\sigma_2\right|^2$. In addition, $\sigma_1, \sigma_2$, are $d$-dimensional vectors of real numbers. The $Z^1$ process governs the conditional mean of the stochastic component to technology growth and the process $Z^2$ captures the exogenous component to aggregate stochastic volatility. Notice that $\sqrt{Z^2}$ scales the Brownian increment to both of the first two processes. The local variance of the exogenous technology shifter is $Z_t^2\left|\sigma_1\right|^2$, and the local variance for the stochastic volatility process is $Z_t^2\left|\sigma_2\right|^2$. The stochastic variance process $Z^2$ is a special cases a Feller square root processes and is normalized to have unconditional mean one. 

## 1.2 Endogenous State Evolution

We next a model with two capital stocks and sluggish adjustment.

$$
\begin{align}
& d K_t^j=K_t^j\left[\Phi_j\left(\frac{I_t^j}{K_t^j}\right)+\beta_j Z_t-\eta_j\right] d t+K_t^j \sqrt{Z_t^2} \sigma_j d B_t
\end{align}
$$

for $j=1,2$. Suppose that output equation is now

$$
\begin{align}
& C_t+I_t^1+I_t^2=\alpha K_t^a
\end{align}
$$

where

$$
\begin{align}
& K_t^a=\left[(1-\zeta)\left(K_t^1\right)^{(1-\kappa)}+\zeta\left(K_t^2\right)^{(1-\kappa)}\right]^{\frac{1}{1-\kappa}}
\end{align}
$$

for $0 \leqslant \zeta<1$.
Form two new state variables. One is $\log Y_t=\log K_t^2-\log K_t^1$ and the other is $\log K_t^a$. where Notice that

$$
\begin{align}
& \frac{K_t^2}{K_t^a} & =\left[(1-\zeta)\left(Y_t\right)^{(\kappa-1)}+\zeta\right]^{\frac{1}{\kappa-1}} \\
& \frac{K_t^1}{K_t^a} & =\left[(1-\zeta)+\zeta\left(Y_t\right)^{(1-\kappa)}\right]^{\frac{1}{\kappa-1}}
\end{align}
$$

Then we have state equations:

$$
\begin{align}
d \log Y_t= & {\left[\Phi_2\left(\frac{I_t^2}{K_t^2}\right)-\Phi_1\left(\frac{I_t^1}{K_t^1}\right)+\left(\beta_2-\beta_1\right) Z_t-\eta_2+\eta_1\right] d t } \\
& -\frac{Z_t^2}{2}\left(\left|\sigma_2\right|^2-\left|\sigma_1\right|^2\right) d t+\sqrt{Z_t^2}\left(\sigma_2-\sigma_1\right) d B_t
\end{align}
$$

$$
\begin{align}
d \log K_t^a= & (1-\zeta)\left(\frac{K_t^1}{K_t^a}\right)^{1-\kappa}\left(\left[\Phi_1\left(\frac{I_t^1}{K_t^1}\right)+\beta_1 Z_t-\eta_1\right] d t+\sqrt{Z_t^2} \sigma_1 d B_t\right) \\
& +\zeta\left(\frac{K_t^2}{K_t^a}\right)^{1-\kappa}\left(\left[\Phi_2\left(\frac{I_t^2}{K_t^2}\right)+\beta_2 Z_t-\eta_2\right] d t+\sqrt{Z_t^2} \sigma_2 d B_t\right) \\
& +\left(\frac{Z_t^2}{2}\right) \operatorname{trace}\left\{\Sigma\left(\widehat{Y}_t\right)\right\} d t
\end{align}
$$

where $\Sigma(y)$ is defined below.

## 1.3 HJB Equation

Rewirte the equation
\begin{equation*}
k_a=\left[(1-\zeta)\left(k_1\right)^{(1-\kappa)}+\zeta\left(k_2\right)^{(1-\kappa)}\right]^{\frac{1}{1-\kappa}}
\end{equation*}

Then the HJB equation is

$$
\begin{split}
0=\max _{i_1, i_2}\Bigg\{&\left(\frac{\delta}{1-\rho}\right)\left(c^{1-\rho} \exp [(\rho-1) v]-1\right) \\
& +(1-\zeta)\left(\frac{k_1}{k_a}\right)^{1-\kappa}\left[\Phi_1\left(i_1\right)+\beta_1 z-\eta_1\right] \\
& +\zeta\left(\frac{k_2}{k_a}\right)^{1-\kappa}\left[\Phi_2\left(i_2\right)+\beta_2 z-\eta_2\right]+\left(\frac{z_2}{2}\right) \text { trace }\left\{\Sigma\left(\frac{k_1}{k_a}, \frac{k_2}{k_a}\right)\right\} \\
& +\frac{\partial v}{\partial \hat{y}}\left[\Phi_2\left(i_2\right)-\Phi_1\left(i_1\right)+\left(\beta_2-\beta_1\right) z-\eta_2+\eta_1-\frac{z_2}{2}\left(\left|\sigma_2\right|^2-\left|\sigma_1\right|^2\right)\right] \\
& +\mu_z \cdot \frac{\partial v}{\partial z}+\frac{z_2}{2} \operatorname{trace}\left\{\left[\begin{array}{cc}
\sigma_2-\sigma_1 \\
\sigma_z
\end{array}\right]^{\prime}\left[\begin{array}{cc}
\frac{\partial^2 v}{\partial \hat{\partial} \partial \hat{y}} & \frac{\partial^2 v}{\partial \hat{y} \partial z^{\prime}} \\
\frac{\partial^2 v}{\partial z^{\prime} \partial \hat{y}} & \frac{\partial^2 v}{\partial z \partial z^{\prime}}
\end{array}\right]\left[\begin{array}{c}
\sigma_2-\sigma_1 \\
\sigma_z
\end{array}\right]\right\} \\
& +\frac{(1-\gamma) z_2}{2}\left|(1-\zeta)\left(\frac{k_1}{k_a}\right)^{1-\kappa} \sigma_1{ }^{\prime}+\zeta\left(\frac{k_2}{k_a}\right)^{1-\kappa} \sigma_2{ }^{\prime}+\left(\sigma_2-\sigma_1\right)^{\prime} \frac{\partial v}{\partial \hat{y}}+\sigma_z{ }^{\prime} \frac{\partial v}{\partial z}\right|^2\Bigg\},
\end{split}
$$

where

$$
\Sigma\left(\frac{k_1}{k_a}, \frac{k_2}{k_a}\right)=(\kappa-1)\left[\begin{array}{ll}
\sigma_1{ }^{\prime} & \sigma_2{ }^{\prime}
\end{array}\right]\left[\begin{array}{cc}
(1-\zeta)^2\left(\frac{k_1}{k_a}\right)^{-2 \kappa+2}-\frac{\kappa}{\kappa-1}(1-\zeta)\left(\frac{k_1}{k_a}\right)^{-\kappa+1} & \zeta(1-\zeta)\left(\frac{k_1}{k_a}\right)^{-\kappa+1}\left(\frac{k_2}{k_a}\right)^{-\kappa+1} \\
\zeta(1-\zeta)\left(\frac{k_1}{k_a}\right)^{-\kappa+1}\left(\frac{k_2}{k_a}\right)^{-\kappa+1} & \zeta^2\left(\frac{k_2}{k_a}\right)^{-2 \kappa+2}-\frac{\kappa}{\kappa-1}(1-\zeta)\left(\frac{k_2}{k_a}\right)^{-\kappa+1}
\end{array}\right]\left[\begin{array}{l}
\sigma_1 \\
\sigma_2
\end{array}\right]
$$

subject to

$$
\begin{align}
c & =\alpha-i_1\left(\frac{k_1}{k_a}\right)-i_2\left(\frac{k_2}{k_a}\right) \\
\frac{k_1}{k_a} & =\left[(1-\zeta)+\zeta y^{1-\kappa}\right]^{\frac{1}{\kappa-1}} \\
\frac{k_2}{k_a} & =\left[(1-\zeta) y^{\kappa-1}+\zeta\right]^{\frac{1}{\kappa-1}}
\end{align}
$$

## 1.4 Quadratic adjustment cost function
$$
\begin{align}
\Phi_i(i)=i-\frac{\phi_i}{2}i^2
\end{align}
$$

this implies

$$
\begin{align}
\Phi_i(i)'=1-{\phi_i}i
\end{align}
$$

## 1.5 First order condition and Hessian Matrix

$$
\begin{align}
\delta \exp [(\rho-1) v] c^{-\rho}(\frac{k_1}{k_a}) &=({1-\phi i_1})[(1-\zeta)\left(\frac{k_1}{k_a}\right)^{1-\kappa}-\frac{\partial v}{\partial \hat{y}}] \\
\delta \exp [(\rho-1) v]c^{-\rho}(\frac{k_2}{k_a})&=({1-\phi i_2})[\zeta\left(\frac{k_2}{k_a}\right)^{1-\kappa}+\frac{\partial v}{\partial \hat{y}}] \\
c &= \alpha-i_1\left(\frac{k_1}{k_a}\right)-i_2\left(\frac{k_2}{k_a}\right)
\end{align}
$$

Hessian Matrix

$$
\begin{align}
    \begin{bmatrix}
-\delta \exp [(\rho-1) v] \rho c^{-\rho-1}(\frac{k_1}{k_a})^2 -\phi_1[(1-\zeta)\left(\frac{k_1}{k_a}\right)^{1-\kappa}-\frac{\partial v}{\partial \hat{y}}]&-\delta \exp [(\rho-1) v] \rho c^{-\rho-1}(\frac{k_1}{k_a}\frac{k_2}{k_a})\\
-\delta \exp [(\rho-1) v] \rho c^{-\rho-1}(\frac{k_1}{k_a}\frac{k_2}{k_a})&-\delta \exp [(\rho-1) v] \rho c^{-\rho-1}(\frac{k_2}{k_a})^2-\phi_2[\zeta\left(\frac{k_2}{k_a}\right)^{1-\kappa}+\frac{\partial v}{\partial \hat{y}}] 
\end{bmatrix}
\end{align}
$$

# 2 Calibration

| Parameter | Value | Alternative |
| --- | --- | --- |
| $\delta$ | 0.002 | |
| $\gamma$ | 1.01, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0 | |
| $\rho$ | 0.67, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5 | |
| $\kappa$ | 0.0, 1.0, 2.0, 4.0, 6.0, 8.0, 10.0 | |
| $\zeta$ | 0.5, 0.333 | |
| $\alpha$ | 0.1 | |
| $\phi_1$ | 28 | |
| $\phi_2$ | 28 | |
| $\eta_1$ | 0.01279 | |
| $\eta_2$ | 0.01279 | |
| $\beta_1$ | 0.01, 0.05 | 0.0 |
| $\beta_2$ | 0.01, 0.05 | |
| $\sigma_{k1}$ | 0.01*sqrt(1.754)*[0.477, 0.0, 0.0, 0.0] | |
| $\sigma_{k2}$ | 0.01*sqrt(1.754)*[0.0, 0.477, 0.0, 0.0] | |
| $a_{11}$ | 0.014 | |
| $a_{22}$ | 0.013 | |
| $\sigma_{z1}$ | [0.011*sqrt(0.5), 0.011*sqrt(0.5), 0.025, 0.0] | |
| $\sigma_{z2}$ | [0.3*sqrt(0.5), 0.3*sqrt(0.5), 0.0, 0.3] | [0.1*sqrt(0.5), 0.1*sqrt(0.5), 0.0, 0.1] |



In [2]:
# Set up model parameters
gamma                = 8.0
rho                  = 1.0
kappa                = 10.0
zeta                 = 0.5
Delta                = 5e-9
symmetric_returns    = 1
dataname             = "Jupyter notebook test"
llim                 = 1.0
srange               = 0.999
lscale               = 0.1
zscale               = 0.1
sscale               = 1.0
preload              = 1
foc                  = 1
clowerlim            = 0.0001
selfpreload          = 0

# Include the Julia support functions 
include("newsets_utils_stk.jl")

println("=============================================================")
# Set up different filename and model type depending on whether returns are symmetric or not
if symmetric_returns == 1
    println(" Economy with two capital stocks: SYMMETRIC RETURNS          ")
    filename = "model_sym_"*string(Delta)*".npz";
elseif symmetric_returns == 0
    println(" Economy with two capital stocks: ASYMMETRIC RETURNS         ")
    filename = "model_asym_"*string(Delta)*".npz";
end

# Construct the path for saving output
filename_ell = "./output/"*dataname*"/llim_"*string(llim)*"_lscale_"*string(lscale)*"_zscale_"*string(zscale)*"/sscale_"*string(sscale)*"_srange_"*string(srange)*"/kappa_"*string(kappa)*"_zeta_"*string(zeta)*"/gamma_"*string(gamma)*"_rho_"*string(rho)*"/"

# Create directory for saving output if it doesn't exist
isdir(filename_ell) || mkpath(filename_ell);

# Assigning various parameters 
a11 = 0.014
a22 = 0.013
alpha = 0.1

# Setting up standard scale and various sigma values
stdscale = sqrt(1.754)
sigma_k1 = stdscale*[.00477, .0, .0, .0]
sigma_k2 = stdscale*[.0, .00477, .0, .0]
sigma_z = [.011*sqrt(.5), .011*sqrt(.5), .025, .0]
sigma_s = [.3*sqrt(.5), .3*sqrt(.5), .0, .3]

# Setting up eta values
eta1 = 0.012790328319261378
eta2 = 0.012790328319261378

# Setting up beta values depending on whether returns are symmetric or not
if symmetric_returns == 1
    beta1 = 0.01
    beta2 = 0.01
else
    beta1 = 0.0
    beta2 = 0.01
end

# Assigning additional parameters
delta = .002
phi1 = 28.0
phi2 = 28.0

# Setting up grid ranges
II, JJ, SS = trunc(Int,1000*lscale+1), trunc(Int,200*zscale+1), trunc(Int,20*sscale+1)
rmax = llim
rmin = -llim
zmax = 1.
zmin = -zmax
smax = 1. + srange
smin = 1. - srange

# Setting up parameters for the numerical solution
maxit = 50000  # maximum number of iterations in the HJB loop
crit  = 10e-6  # convergence criterion HJB loop

# Initialize model objects
# Create two Baseline objects, one for each capital stock
baseline1 = Baseline(a11, a22, zeta, kappa, sigma_z, sigma_s, beta1, eta1, sigma_k1, delta)
baseline2 = Baseline(a11, a22, zeta, kappa, sigma_z, sigma_s, beta2, eta2, sigma_k2, delta)

# Create two Technology objects, one for each capital stock
technology1 = Technology(alpha, phi1)
technology2 = Technology(alpha, phi2)

# Create the TwoCapitalEconomy model using the baseline and technology objects
model = TwoCapitalEconomy(baseline1, baseline2, technology1, technology2)

# Create the Grid_rz object for the state space
grid = Grid_rz(rmin, rmax, II, zmin, zmax, JJ, smin, smax, SS)

# Create the FinDiffMethod object for the finite difference method parameters
params = FinDiffMethod(maxit, crit, Delta);


 Economy with two capital stocks: SYMMETRIC RETURNS          


# 3 Solve for the HJB equations

In [3]:
# Define the preload parameters 
preload_Delta = 150.0
preload_delta = delta
preload_zeta = zeta

if kappa == 0.0
    preload_llim = 18.0
else
    preload_llim = 1.0
end

preload_lscale = 3.0
preload_zscale = 1.0
preload_kappa = kappa
preload_zeta = zeta
preload_rho = rho
preload_gamma = gamma

# Construct the path to preload file depending on the preload parameters
preloadname = (
    "/project/lhansen/twocapstc/output/" * 
    "twocap_large_scale" * 
    "/Delta_" * string(preload_Delta) * 
    "_llim_" * string(preload_llim) * 
    "_lscale_" * string(preload_lscale) * 
    "_zscale_" * string(preload_zscale) *
    "/delta_" * string(preload_delta) *
    "/kappa_" * string(preload_kappa) * 
    "_zeta_" * string(preload_zeta) * 
    "/gamma_" * string(gamma) * 
    "_rho_" * string(preload_rho) * "/"
)

if symmetric_returns == 1
    preload = npzread(preloadname*"model_sym_HS.npz")
else
    preload = npzread(preloadname*"model_asym_HS.npz")
end
println("preload location : "*preloadname)

# Create ranges for A_x1, A_x2, A_rr, A_zz
A_x1 = range(-llim,llim,trunc(Int,1000*preload_lscale+1))
A_x2 = range(-1,1,trunc(Int,200*preload_zscale+1))
A_rr = range(rmin, stop=rmax, length=II);
A_zz = range(zmin, stop=zmax, length=JJ);

# Preload the initial guess for the value function from the two-dimensional model
# The code interpolates the two-dimensional results onto the three-dimensional grid
println("preload V0 starts")
preloadV0 = ones(II, JJ, SS)
itp = interpolate(preload["V"], BSpline(Cubic(Line(OnGrid()))))
sitp = scale(itp, A_x1, A_x2)
println("(1,-1): ",sitp(1,-1))
preloadV0tdm = ones(II, JJ)
for i = 1:II
    for j = 1:JJ
        preloadV0tdm[i,j] = sitp(A_rr[i], A_zz[j])
    end
end
println("(1,-1): ",preloadV0tdm[end,1])
for i = 1:SS
    preloadV0[:,:,i] = preloadV0tdm
end
println("preload V0 ends")
println("preload d1 starts")
preloadd1 = ones(II, JJ, SS)
itp = interpolate(preload["d1"], BSpline(Cubic(Line(OnGrid()))))
sitp = scale(itp, A_x1, A_x2)
println("(1,-1): ",sitp(1,-1))
preloadd1tdm = ones(II, JJ)
for i = 1:II
    for j = 1:JJ
        preloadd1tdm[i,j] = sitp(A_rr[i], A_zz[j])
    end
end
println("(1,-1): ",preloadd1tdm[end,1])
for i = 1:SS
    preloadd1[:,:,i] = preloadd1tdm
end
println("preload d1 ends")
println("preload cons starts")
preloadcons = ones(II, JJ, SS)
itp = interpolate(preload["cons"], BSpline(Cubic(Line(OnGrid()))))
sitp = scale(itp, A_x1, A_x2)
println("(1,-1): ",sitp(1,-1))
preloadconstdm = ones(II, JJ)
for i = 1:II
    for j = 1:JJ
        preloadconstdm[i,j] = sitp(A_rr[i], A_zz[j])
    end
end
println("(1,-1): ",preloadconstdm[end,1])
for i = 1:SS
    preloadcons[:,:,i] = preloadconstdm
end
println("preload cons ends")
println("preload VrF starts")
preloadVrF = ones(II, JJ, SS)
itp = interpolate(preload["Vr_F"], BSpline(Cubic(Line(OnGrid()))))
sitp = scale(itp, A_x1, A_x2)
println("(1,-1): ",sitp(1,-1))
preloadVrFtdm = ones(II, JJ)
for i = 1:II
    for j = 1:JJ
        preloadVrFtdm[i,j] = sitp(A_rr[i], A_zz[j])
    end
end
println("(1,-1): ",preloadVrFtdm[end,1])    
for i = 1:SS
    preloadVrF[:,:,i] = preloadVrFtdm
end
println("preload VrF ends")
println("preload VrB starts")
preloadVrB = ones(II, JJ, SS)
itp = interpolate(preload["Vr_B"], BSpline(Cubic(Line(OnGrid()))))
sitp = scale(itp, A_x1, A_x2)
println("(1,-1): ",sitp(1,-1))
preloadVrBtdm = ones(II, JJ)
for i = 1:II
    for j = 1:JJ
        preloadVrBtdm[i,j] = sitp(A_rr[i], A_zz[j])
    end
end
println("(1,-1): ",preloadVrBtdm[end,1])
for i = 1:SS
    preloadVrB[:,:,i] = preloadVrBtdm
end
println("preload Vr ends")

preload location : /project/lhansen/twocapstc/output/twocap_large_scale/Delta_150.0_llim_1.0_lscale_3.0_zscale_1.0/delta_0.002/kappa_10.0_zeta_0.5/gamma_8.0_rho_1.0/
preload V0 starts
(1,-1): -2.1169990054969814
(1,-1): -2.1169990054969814
preload V0 ends
preload d1 starts
(1,-1): 0.03508224782840131
(1,-1): 0.03508224782840131
preload d1 ends
preload cons starts
(1,-1): 0.98140992486655
(1,-1): 0.98140992486655
preload cons ends
preload VrF starts
(1,-1): -1.0408340855860843e-17
(1,-1): -1.0408340855860843e-17
preload VrF ends
preload VrB starts
(1,-1): 0.0937456293073069
(1,-1): 0.0937456293073069
preload Vr ends


In [4]:
## Use the Finite Difference Method to solve the model
times = @elapsed begin
A, V, val, d1_F, d2_F, d1_B, d2_B, h1_F, h2_F, hz_F, hs_F, h1_B, h2_B, hz_B, hs_B,
            mu_1_F, mu_1_B, mu_r_F, mu_r_B, mu_z, mu_s, V0, Vr, Vz, Vs, Vr_F, Vr_B, Vz_B, Vz_F, Vs_B, Vs_F, cF, cB, rrr, zzz, sss, pii, dr, dz, ds =
        value_function_twocapitals(gamma, rho, model, grid, params, preloadV0, preloadd1, preloadcons, preloadVrF, preloadVrB, foc, clowerlim, symmetric_returns);
println("=============================================================")
end
println("Convegence time (minutes): ", times/60)

## Solve for the stationary distribution
g = stationary_distribution(A, grid);

rmax = 1.0, rmin = -1.0, rlength = 101
zmax = 1.0, zmin = -1.0, zlength = 21
smax = 1.999, smin = 0.0010000000000000009, slength = 21
----------------------------------
Iteration = 1
Distance = 2.6558001353649274e-9
v max = -0.9021939670433878
v min = -2.1897813509118422
cF max = 1.9034491962810565
cF min = 0.03641740914014571
cB max = 1.9034491962810565
cB min = 0.036417413546907744
d1 max = 0.034646947504492334
d1 min = -0.0014445554796030935
d1F max = 0.03567953670358138
d1F min = -0.0014445554796030935
d1B max = 0.034646947504492334
d1B min = -0.7296836099681218
d2 max = 0.03472200149372322
d2 min = 0.00039291638044280236
d2F max = 0.03472200149372322
d2F min = -0.7296836099681218
d2B max = 0.03567953670358138
d2B min = 0.00039291638044280236
h1 max = 0.010707935290455524
h1 min = 0.0037635223988825442
h2 max = 0.01075113772446418
h2 min = 0.0038067248328940866
hz max = 0.015624955198254353
hz min = 0.010722279948104485
hs max = 0.0
hs min = 0.0
----------------------------------
 

In [5]:
## Store All the results in a dictionary

policies  = PolicyFunctions(d1_F, d2_F, d1_B, d2_B,
                            -h1_F, -h2_F, -hz_F, -hs_F,
                            -h1_B, -h2_B, -hz_B, -hs_B);

mu_1 = (mu_1_F + mu_1_B)/2.;
mu_r = (mu_r_F + mu_r_B)/2.;
h1_dist = (policies.h1_F + policies.h1_B)/2.;
h2_dist = (policies.h2_F + policies.h2_B)/2.;
hz_dist = (policies.hz_F + policies.hz_B)/2.;
hs_dist = (policies.hs_F + policies.hs_B)/2.;
d1 = (policies.d1_F + policies.d1_B)/2;
d2 = (policies.d2_F + policies.d2_B)/2;
h1, h2, hz, hs= -h1_dist, -h2_dist, -hz_dist, -hs_dist;

r = range(rmin, stop=rmax, length=II);   
rr = r * ones(1, JJ);
rrr = ones(II, JJ, SS)
for i = 1:SS
    rrr[:,:,i] = rr
end
pii = rrr;
IJS = II*JJ*SS;
k1a = zeros(II,JJ,SS)
k2a = zeros(II,JJ,SS)
for i=1:IJS
    p = pii[i];
    k1a[i] = (1-zeta + zeta*exp.(p*(1-kappa))).^(1/(kappa-1));
    k2a[i] = ((1-zeta)*exp.(p*((kappa-1))) + zeta).^(1/(kappa-1));
end
d1k = d1.*k1a
d2k = d2.*k2a
c = alpha*ones(II,JJ,SS) - d1k - d2k

results = Dict("delta" => delta,
"eta1" => eta1, "eta2" => eta2, "a11"=> a11, "a22"=> a22, 
"beta1" => beta1, "beta2" => beta2, "k1a" => k1a, "k2a"=> k2a, "d1k" => d1k, "d2k"=> d2k,
"sigma_k1" => sigma_k1, "sigma_k2" => sigma_k2,
"sigma_z" =>  sigma_z, "sigma_s" =>  sigma_s, "alpha" => alpha, "kappa" => kappa, "zeta" => zeta, "phi1" => phi1, "phi2" => phi2,
"I" => II, "J" => JJ, "S" => SS, "rmax" => rmax, "rmin" => rmin, "zmax" => zmax, "zmin" => zmin,
"rrr" => rrr, "zzz" => zzz, "sss" => sss, "pii" => pii, "dr" => dr, "dz" => dz,
"maxit" => maxit, "crit" => crit, "Delta" => Delta, "g" => g, "cons" => c,
"V0" => V0, "V" => V, "Vr" => Vr, "Vz" => Vz, "Vs" => Vs, "Vr_F" => Vr_F, "Vr_B" => Vr_B,  "Vz_F" => Vz_F, "Vz_B" => Vz_B, "Vs_F" => Vs_F, "Vs_B" => Vs_B, "val" => val, "gamma" => gamma, "rho" => rho,
"d1" => d1, "d2" => d2, "d1_F" => d1_F, "d2_F" => d2_F, "d1_B" => d1_B, "d2_B" => d2_B, "cF" => cF, "cB" => cB,
"h1_F" => h1_F, "h2_F" => h2_F, "hz_F" => hz_F, "hs_F" => hs_F, "h1_B" => h1_B, "h2_B" => h2_B, "hz_B" => hz_B, "hs_B" => hs_B, 
"mu_1_F" => mu_1_F, "mu_1_B" => mu_1_B, "mu_r_F" => mu_r_F, "mu_r_B" => mu_r_B, 
"h1" => h1, "h2" => h2, "hz" => hz, "hs" => hs, "foc" => foc, "clowerlim" => clowerlim,  "zscale" => zscale, "lscale" => lscale, "sscale" => sscale, "llim" => llim,"srange" => srange,
"times" => times,
"mu_1" => mu_1, "mu_r" => mu_r, "mu_z" => mu_z, "mu_s" => mu_s)

npzwrite(filename_ell*filename, results)
